In [ ]:
# Analysis of HUDOC documents
# Corpus from 31/01/2019, scraped using https://github.com/ajbarker93/caseScrape

In [ ]:
# Import libraries
import json
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from price_parser import Price
from sklearn.linear_model import LinearRegression
from forex_python.converter import CurrencyRates
import seaborn as sns
import datefinder

In [ ]:
# Check drive contents
rootdir="/users/adambarker/case_scrape/"
os.chdir(rootdir)
os.listdir()

In [ ]:
# Import pickle file of results content
results = pd.read_pickle("./results0.pkl")

# Remove all rows where the content failed
results = results.loc[results['content']!='']

In [ ]:
# Append violations

reload = 1
if reload:
    # Add additional fields
    results['violation'] = ''
    results['conclusion'] = ''
    results['importance'] = ''
    results['originatingbody'] = '' 
    results['case_date'] = ''

    # Load summaries
    onlyfiles = [f for f in listdir(rootdir+"/summaries/") if isfile(join(rootdir+"/summaries/", f))]
    onlyjsons = [f for f in onlyfiles if f.endswith('.json')]

    # Examine summaries and choose the list of P1-1 relevant cases
    for onlyjson in onlyjsons:
        
        try:
            
            with open(rootdir+"/summaries/"+onlyjson) as json_file: # first open the batches of summaries

                contents = json.load(json_file)
                contents2 = contents['results']

                for i in range(0,len(contents2)): # loop through each case in the batch

                    field_data = contents2[i]['columns']

                    if field_data['itemid'] in results.case_id.values:

                        results.loc[results['case_id'] == field_data['itemid'],['violation']] = field_data['violation']
                        results.loc[results['case_id'] == field_data['itemid'],['conclusion']] = field_data['conclusion']
                        results.loc[results['case_id'] == field_data['itemid'],['importance']] = field_data['importance']
                        results.loc[results['case_id'] == field_data['itemid'],['originatingbody']] = field_data['originatingbody']

                        
        except:
            print("Error")
            
    # Save as pickle again for easy recall
    results.to_pickle("./results.pkl")    

In [ ]:
# Inspect some content
results.head()

In [ ]:
# This is a loading script which does all of the io on the directory. It contains:
# Strings to define start and end of relevant sections

reload = 1
if reload:

    # Define a simple set of start and end wildcards for the claimed and awarded sections
    cl_start_str='SECTION'
    cl_end_str='President'
    aw_start_str='SECTION'
    aw_end_str='President'

    # First do English cases
    for i in range(0,len(results)):

        result = results['content'].iloc[i]

        if len(result.split(cl_start_str))>1:

            # Choose text from start wildcard onwards
            to_end = result.split(cl_start_str)[1]
            # Choose text before end wildcard
            from_start = to_end.split(cl_end_str)[0]
            # Add to resuls array
            results['claimed'].iloc[i] = from_start

        if len(result.split(aw_start_str))>1:

            # Repeat for awarded section
            to_end = result.split(aw_start_str)[1]
            from_start = to_end.split(aw_end_str)[0]
            results['awarded'].iloc[i] = from_start
                
    # Then do French cases
    cl_start_str='Note d’information'
    cl_end_str='Cliquez ici pour accéder aux'
    aw_start_str='Note d’information'
    aw_end_str='Cliquez ici pour accéder aux'

    for i in range(0,len(results)):

        if results['claimed'].iloc[i] =='':

            result = results['content'].iloc[i]

            if len(result.split(cl_start_str))>1:

                # Choose text from start wildcard onwards
                to_end = result.split(cl_start_str)[1]
                # Choose text before end wildcard
                from_start = to_end.split(cl_end_str)[0]
                # Add to resuls array
                results['claimed'].iloc[i] = from_start

            if len(result.split(aw_start_str))>1:

                # Repeat for awarded section
                to_end = result.split(aw_start_str)[1]
                from_start = to_end.split(aw_end_str)[0]
                results['awarded'].iloc[i] = from_start         
            
    # Save as pickle again for easy recall
    results.to_pickle("./results.pkl")

In [ ]:
# Get case dates

try: 
    for i in range(0,len(results)):

        # Get dates while we're here look at first 40 words
        headline = result.split("JUDGMENT")
        if len(headline)>1:

            specific_split = headline[1].split()
            split_depth = min(20,len(specific_split))
            date_str = ' '.join(specific_split[:split_depth])
            matches = list(datefinder.find_dates(date_str))

            if len(matches) > 1:
                # date returned will be a datetime.datetime object. here we are only using the first match.
                results['case_date'].iloc[i] = matches[1]   
            elif len(matches)==1:
                results['case_date'].iloc[i] = matches[0]


        # Get dates while we're here look at first 40 words
        headline = result.split("Arrêt")
        if len(headline)>1:
            specific_split = headline[1].split()
            split_depth = min(20,len(specific_split))
            date_str = ' '.join(specific_split[:split_depth])
            matches = list(datefinder.find_dates(date_str))
            if len(matches) > 1:
                # date returned will be a datetime.datetime object. here we are only using the first match.
                results['case_date'].iloc[i] = matches[1]   
            elif len(matches)==1:
                results['case_date'].iloc[i] = matches[0]  
                
except:
    print("Error")

In [ ]:
#results['case_date'] = ''
results.head()

In [ ]:
# Results

# Load pickle if required
reload = 0
if reload:
    results = pd.read_pickle("./results.pkl") 

results.head()

In [ ]:
# Now find numerical content and try to arrive at a claimed figure

# Define list of allowed currencies
curr = ['EUR','EURO','GBP','USD','Euros','Pounds','Dollars','French','Turkish','FRF','Lira','Turkish Lira']

# Remove all rows where the content failed
results = results.loc[results['claimed']!='']
results = results.loc[results['awarded']!='']

# Define function to find currencies in string
def find_currency(x):
    px = [Price.fromstring(i) for i in x.split(".")]
    px2 = [t.amount for t in px if t.currency in curr]
    return px2

# Update the num claimed col
results['num_claimed'] = results.apply(lambda x: find_currency(str(x['claimed'])),axis=1)
results['num_awarded'] = results.apply(lambda x: find_currency(str(x['awarded'])),axis=1)

# Set the fields to zero if they contain an empty list
results.num_awarded = results.num_awarded.apply(lambda y: [0] if (len(y)==0) else y)
results.num_claimed = results.num_claimed.apply(lambda y: [0] if (len(y)==0) else y)

In [ ]:
# Remove all rows where the content failed
results = results.loc[results['violation']!='']

# Inspect results
results.head(10)

In [ ]:
# Set wildcards for selecting P1-1 cases
wildcards_p1 = ['1-1','P1','P1-1','P1-1-1','Protocol 1-1','Article 1','Article 1 al. 1 du Protocole','1 du Protocole n° 1','1-2']
wildcards_p2 = ['P2','2-1']
wildcards_p3 = ['P3']
wildcards_p4 = ['P4','4-2']
wildcards_p5 = ['P5','5-2','5-3','5-4','5-1','5-5']
wildcards_p6 = ['P6','6-1','6-3','6-2']
wildcards_p8 = ['P8','8-1']
wildcards_p13 = ['13-P4','13','13+3','13+7','13+5']

# Do filtering into various articles
results['article'] = ''
results['article'] = results.apply(lambda x: 'p1' if any(i in x['violation'] for i in wildcards_p1) else x['article'],axis=1)
results['article'] = results.apply(lambda x: 'p2' if any(i in x['violation'] for i in wildcards_p2) else x['article'],axis=1)
results['article'] = results.apply(lambda x: 'p4' if any(i in x['violation'] for i in wildcards_p4) else x['article'],axis=1)
results['article'] = results.apply(lambda x: 'p5' if any(i in x['violation'] for i in wildcards_p5) else x['article'],axis=1)
results['article'] = results.apply(lambda x: 'p6' if any(i in x['violation'] for i in wildcards_p6) else x['article'],axis=1)
results['article'] = results.apply(lambda x: 'p8' if any(i in x['violation'] for i in wildcards_p8) else x['article'],axis=1)
results['article'] = results.apply(lambda x: 'p13' if any(i in x['violation'] for i in wildcards_p13) else x['article'],axis=1)

In [ ]:
# Define originator
results['originator'] = ''
results['defendant'] = ''

try:
    # Split to get originator and defendant
    # Source
    results['originator'] = results.apply(lambda x: x['case'].split(" v.")[0].split(" ")[-1] if len(x['case'].split(" v."))>1 else x['originator'],axis=1)
    results['originator'] = results.apply(lambda x: x['case'].split(" c.")[0].split(" ")[-1] if len(x['case'].split(" c."))>1 else x['originator'],axis=1)
    # Defendant
    results['defendant'] = results.apply(lambda x: x['case'].split(" v. ")[1].split(" ")[0] if len(x['case'].split(" v. "))>1 else x['defendant'],axis=1)
    results['defendant'] = results.apply(lambda x: x['case'].split(" c. ")[1].split(" ")[0] if len(x['case'].split(" c. "))>1 else x['defendant'],axis=1)

    # Title format the originator and defendant
    results['originator'] = results.apply(lambda x: x['originator'].title(),axis=1)
    results['defendant'] = results.apply(lambda x: x['defendant'].title(),axis=1)

    # Add the first name
    results['originator'] = results.apply(lambda x: x['content'].split(str(x['originator']))[0].split(" ")[-1] + " " + x['originator'] if len(x['originator'])>1 else x['originator'],axis=1)
    
    # Then look in text for prefix or suffix
    results['type'] = ''
    results['type'] = results.apply(lambda x: 'M' if 'Mr '+x['originator'] in x['content'] else x['type'],axis=1)
    results['type'] = results.apply(lambda x: 'F' if 'Mrs '+x['originator'] in x['content'] else x['type'],axis=1)

except:
    print("Error")

results.head(5)

In [ ]:
# Save as pickle again for easy recall
results.to_pickle("./results.pkl")